Authors: Zhewei Yao <https://github.com/yaozhewei>, Amir Gholami <http://amirgholami.org/>


This tutorial shows how to compute the Hessian information using (randomized) numerical linear algebra for both explicit Hessian (the matrix is given) as well as implicit Hessian (the matrix is ungiven).

We'll start by doing the necessary imports:

In [1]:
import numpy as np
import torch 
from torchvision import datasets, transforms
from utils import * # get the dataset
from pyhessian import hessian
from pyhessian.hessian_with_activation import hessian_with_activation # Hessian computation
from density_plot import get_esd_plot # ESD plot
from pytorchcv.model_provider import get_model as ptcv_get_model # model
from pyhessian.utils import group_product

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# enable cuda devices
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# device = torch.device("cuda:0,1")

In [3]:
# get the model 
model = ptcv_get_model("resnet20_cifar10", pretrained=True)

# change the model to eval mode to disable running stats upate
model.eval()


CIFARResNet(
  (features): Sequential(
    (init_block): ConvBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU(inplace=True)
    )
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBlock(
          (conv1): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): ReLU(inplace=True)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (activ): ReLU(inplace=True)
      )
      (unit2): ResUnit(
        (body): ResBlock(
          (co

In [4]:

# create loss function
criterion = torch.nn.CrossEntropyLoss()

# get dataset 
train_loader, test_loader = getData(train_bs=20, train_length=0.02)

model = model.cuda()

# for illustrate, we only use one batch to do the tutorial
for inputs, targets in train_loader:
    break;
print(len(train_loader))    
print(len(inputs))


Files already downloaded and verified
50
20


In [5]:
# create the hessian computation module
hessian_comp = hessian_with_activation(model, criterion, dataloader=train_loader, cuda=True)
hessian_comp.insert_hook("conv")

In [6]:
# v = hessian_comp.get_activ_rand_v()
hessian_comp.get_activ_rand_v(show_layer=True, dont_reset=True)
for layer in hessian_comp.activation_grads.keys():
    has_grad = hessian_comp.activation_grads[layer][0] #.require_grad()
    # print(dir(has_grad))
    if (has_grad is not None):
        print(f"*** {layer} : {has_grad.grad_fn}")

append features.stage1.unit1.body.conv1.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage1.unit1.body.conv2.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage1.unit2.body.conv1.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage1.unit2.body.conv2.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage1.unit3.body.conv1.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage1.unit3.body.conv2.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage2.unit1.identity_conv.conv, active size : torch.Size([20, 16, 32, 32]), active grad size : torch.Size([20, 16, 32, 32])
append features.stage2.unit1.bo

In [7]:
act_trace = hessian_comp.trace_activ(maxIter=50, tol=1e-6)
print(act_trace[-2])
print(act_trace[-1])

KeyboardInterrupt: 

In [12]:
print((act_trace[-2]))
print(act_trace[-1])

[27.796939849853516, 2.0737404823303223, 0.19290420413017273, 1.0326926708221436, 0.1409032791852951, 1.496253490447998, 0.08517349511384964, 0.19383658468723297, 0.5825191736221313, 0.10495015233755112, 0.9547169804573059, 0.09460827708244324, 2.080573320388794, 0.023283323273062706, 0.2941017150878906, 2.0656158924102783, 0.8748257756233215, 1.6801472902297974, 0.4482768177986145, 0.389938086271286]
[11.472389221191406, 1.8090567588806152, 0.26039567589759827, 1.5163310766220093, 0.09185336530208588, 0.681473970413208, 0.08558680862188339, 0.06237340718507767, 1.7694450616836548, 0.08589757233858109, 1.5487838983535767, 0.18010348081588745, 2.0904734134674072, 0.03921247273683548, 0.3117707371711731, 1.1553516387939453, 1.6646945476531982, 2.620756149291992, 1.0674006938934326, 0.5741992592811584]


In [6]:
trace = hessian_comp.trace(maxIter=1)
print("The trace of this model is: %.4f"%(np.mean(trace)))

# print(hessian_comp.activations.keys())

check dataload hv product
The trace of this model is: 296.7292


In [7]:

# # print(hessian_comp.activations.keys())
# print((hessian_comp.activation_grads["features.stage1.unit1.body.conv1.conv"][0][0].size()))
# print((hessian_comp.activations["features.stage1.unit1.body.conv1.conv"][0][0].size()))
# # print((hessian_comp.activations["features.stage1.unit1.body.conv1.conv"][0][0].grad))
# print((hessian_comp.activations["features.stage1.unit1.body.conv1.conv"][0][0]))
# print((hessian_comp.activation_grads["features.stage1.unit1.body.conv1.conv"][0][0]))
# # print((hessian_comp.activation_grads["features.stage1.unit1.body.conv1.conv"][0][0].grad_fn.next_functions))

In [9]:
for layer in hessian_comp.activations.keys():
    input_size = torch.randint_like(hessian_comp.activations[layer][0][0], high=2, device="cuda").size()
    if hessian_comp.activation_grads[layer][0][0] is not None:
        grad_size = hessian_comp.activation_grads[layer][0][0].size()
    else:
        grad_size = 1
    # print(type(hessian_comp.activation_grads[layer][i][0]))
    if(input_size != grad_size):
        print(f"************* {layer} not equal!! ************")
        print( input_size, grad_size, "\n\n")
    else:
        print(f"************* {layer} ************")
        print( input_size, "\n\n")


************* features.init_block.conv ************
torch.Size([5, 3, 32, 32]) 1 


************* features.stage1.unit1.body.conv1.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage1.unit1.body.conv2.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage1.unit2.body.conv1.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage1.unit2.body.conv2.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage1.unit3.body.conv1.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage1.unit3.body.conv2.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage2.unit1.identity_conv.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.stage2.unit1.identity_conv ************
torch.Size([5, 16, 32, 32]) torch.Size([5, 32, 16, 16]) 


************* features.stage2.unit1.body.conv1.conv ************
torch.Size([5, 16, 32, 32]) 


************* features.

In [9]:
"""
hessian_comp.activations["features.stage3.unit2.activ"] are saved inputs of each layer
while executing trace and dataload_hv_product for loop. 
They listed by trace and dataload_hv_product for loop.

Dim 1 of hessian_comp.activations["features.stage3.unit2.activ"][0] means batch.
"""
# v = torch.randint_like(hessian_comp.activations["features.stage1.unit1.body.conv1.conv"][0][0], high=2, device="cuda")
rand_vs = []
activ_grads = []
activs = []

for layer in hessian_comp.activations.keys():
    for i in range(len(hessian_comp.activations[layer])):
        activ_element = hessian_comp.activations[layer][i][0]
        grad_element = hessian_comp.activation_grads[layer][i][0]

        if (grad_element is None):
            continue
        elif grad_element.size() != activ_element.size() :
            continue
        else:    
            rand_vs.append(torch.randint_like(hessian_comp.activations[layer][i][0], high=2, device="cuda"))
            activ_grads.append(hessian_comp.activation_grads[layer][i][0])
            activs.append(hessian_comp.activations[layer][i][0])


Hv_list = []
trace_list = []
for (v, grad, activ) in zip(rand_vs, activ_grads, activs):
    Hv = torch.autograd.grad(
        grad, 
        activ, 
        grad_outputs=v, only_inputs=True, retain_graph=True)
    Hv_list.append(Hv)
    trace_list.append(group_product(Hv, v).cpu().item())

print(trace_list)


# Hv = torch.autograd.grad(
#     hessian_comp.activation_grads["features.stage1.unit1.body.conv1.conv"][0][0], 
#     hessian_comp.activations["features.stage1.unit1.body.conv1.conv"][0][0], 
#     grad_outputs=v, only_inputs=True, retain_graph=False)



# a = torch.autograd.grad(hessian_comp.activation_grads["features.stage3.unit2.activ"][1][0], hessian_comp.activations["features.stage3.unit2.activ"][1]) 
# print(hessian_comp.activations["features.stage3.unit2.activ"][1].size())

[0.2398936152458191, 0.2883727252483368, 0.06399604678153992, 2.021277904510498, 0.024379881098866463, 2.3017399311065674, 0.05182919651269913, 0.07555367797613144, 0.4853941798210144, 0.007302280515432358, 0.8529448509216309, 0.2646264433860779, 21.458599090576172, 0.010562198236584663, 0.03980473428964615, 3.2707457542419434, 1.180624008178711, 5.310753345489502, 2.88645339012146, 5.8211669921875]


In [12]:
layer1 = "features.stage1.unit1.body.conv1.conv"

tmp_v = torch.randint_like(hessian_comp.activations[layer1][0][0], high=2, device="cuda")


In [19]:
layer2 = "features.stage1.unit1.body.conv1.conv"

Hv = torch.autograd.grad(
    hessian_comp.activation_grads[layer1][0][0], 
    hessian_comp.activations[layer2][0][0], 
    grad_outputs=tmp_v, only_inputs=True, retain_graph=True)



print(group_product(Hv, tmp_v).cpu().item())


0.6590860486030579


## Example 1: Power Iteration with Numpy

The following part shows how to use power iteration to get the top eigenvalue of a matrix without explicitly having access to it in numpy. We start by creating a random matrix B, compute its ground truth eigenvalues using numpy, and then compare the results with matrix-free power iteration (which does not need direct access to the matrix).

In [7]:
n = 10 # the matrix size

# generate a random matrix
A = np.random.randn(n, n)
B = A @ A.T

Now let's use numpy to compute the ground truth eigenvalues. We will then check the results with this.

In [8]:
# use np.eigs to get the top eigenvalue of B
eigs, _ = np.linalg.eig(B)

print("The top eigenvalue of B is %.4f"%np.sort(eigs)[-1])

The top eigenvalue of B is 21.3926


Now let's try to comptue the top eigenvalue of B without explicitly accessing B. To do so, we will use a method called Power Iteration:

https://en.wikipedia.org/wiki/Power_iteration

The algorithm is very simple and efficiet to compute the top eigenvalue:
$$v_{i+1} = \frac{Bv_i}{\|Bv_i\|}.$$

As such, we only need to have access to the *application* of B to a given vector $v_i$ and not the matrix B itself. This application is commonly referred to as *matvec* in literature.

In [9]:
# use power iteration to get the top eigenvalue of B
v = np.random.randn(n, 1)
for i in range(40):
    v = v / np.linalg.norm(v)
    eig_power_iteration = v.T @ B @ v
    print("Step %2d current estimated top eigvalue: %.4f"%(i+1,eig_power_iteration))
    v = B @ v
print("Finished Power Iteration\n")
print("Ground Truth Top Eigenvalue: %.4f"%np.sort(eigs)[-1])
print("Result with matrix-free Power Iteration: %.4f"%eig_power_iteration)

Step  1 current estimated top eigvalue: 10.7443
Step  2 current estimated top eigvalue: 19.2796
Step  3 current estimated top eigvalue: 20.0942
Step  4 current estimated top eigvalue: 20.5082
Step  5 current estimated top eigvalue: 20.8005
Step  6 current estimated top eigvalue: 21.0048
Step  7 current estimated top eigvalue: 21.1427
Step  8 current estimated top eigvalue: 21.2333
Step  9 current estimated top eigvalue: 21.2918
Step 10 current estimated top eigvalue: 21.3291
Step 11 current estimated top eigvalue: 21.3527
Step 12 current estimated top eigvalue: 21.3676
Step 13 current estimated top eigvalue: 21.3770
Step 14 current estimated top eigvalue: 21.3828
Step 15 current estimated top eigvalue: 21.3865
Step 16 current estimated top eigvalue: 21.3888
Step 17 current estimated top eigvalue: 21.3902
Step 18 current estimated top eigvalue: 21.3911
Step 19 current estimated top eigvalue: 21.3917
Step 20 current estimated top eigvalue: 21.3920
Step 21 current estimated top eigvalue: 

As you can see the result of the power iteration and the one we got from numpy match very well.

We can apply the same techinique for neural networks as well, and in particular use it to compute eigenvalues of Hessian!

Importantly there has been a lot of misconception that we can not use Hessian for real world applications since
we need to explicitly form it. Next you will see that this is not correct.

## Example 2: Power Iteration for NN Hessian

In [10]:
# get the model 
model = ptcv_get_model("resnet20_cifar10", pretrained=True)
# change the model to eval mode to disable running stats upate
model.eval()

# create loss function
criterion = torch.nn.CrossEntropyLoss()

# get dataset 
train_loader, test_loader = getData()

# for illustrate, we only use one batch to do the tutorial
for inputs, targets in train_loader:
    break

# we use cuda to make the computation fast
model = model.cuda()
inputs, targets = inputs.cuda(), targets.cuda()



Files already downloaded and verified


In [11]:
# create the hessian computation module
hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

In [12]:
# Now let's compute the top eigenvalue. This only takes a few seconds.
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()
print("The top Hessian eigenvalue of this model is %.4f"%top_eigenvalues[-1])

The top Hessian eigenvalue of this model is 147.6936


In [13]:
# Now let's compute the top 2 eigenavlues and eigenvectors of the Hessian
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(top_n=2)
print("The top two eigenvalues of this model are: %.4f %.4f"% (top_eigenvalues[-1],top_eigenvalues[-2]))

The top two eigenvalues of this model are: 147.6954 125.3500


The small difference between this top eigenvalue (195.4954) and the previous one (195.5897) is due to the small number of iterations that we used in Power iteration. You can remove this small difference by increasing the number of iterations for power iteration.

## Example 2.1: Plot Loss Landscape

We can use the Hessian eigenvectors/eigenvalues to analyze the flat/sharpness of the loss landscape of your model, and plot the loss landscape. We will show that this can be more informative than using random directions.

To plot the loss landscape, we first compute the top Hessian eigenvector and then perturb the model parameters along that direction and measure the loss.

In [ ]:
# get the top eigenvector
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()

In [ ]:
# This is a simple function, that will allow us to perturb the model paramters and get the result
def get_params(model_orig,  model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(model_orig.parameters(), model_perb.parameters(), direction):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

In [ ]:
# lambda is a small scalar that we use to perturb the model parameters along the eigenvectors 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

for lam in lams:
    model_perb = get_params(model, model_perb, top_eigenvector[0], lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')
plt.title('Loss landscape perturbed based on top Hessian eigenvector')

Now let's compare this with a loss landscape computed based on perturbing the model parameters along a random direction.

In [ ]:
from pyhessian.utils import normalization

# generate random vector to do the loss plot

v = [torch.randn_like(p) for p in model.parameters()]
v = normalization(v)


# used to perturb your model 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

for lam in lams: 
    model_perb = get_params(model, model_perb, v, lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')
plt.title('Loss landscape perturbed based on a random direction')

Note how different the loss landscape looks. In particular note that there is almost no change in the loss value (see the small scale of the y-axis). This is expected, since for a converged NN, many of the directions are typically degenarate (i.e. they are flat).

We can also use gradient direction to perturb the model. While gradient is better than random vector, but it is not possible to use it to plot 3D loss landscape since you will need more than one direction. However, you can use top 2 Hessian vectors instead for that scenario.

In [ ]:
from pyhessian.utils import normalization


# used to perturb your model 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

# generate gradient vector to do the loss plot
loss = criterion(model_perb(inputs), targets)
loss.backward()

v = [p.grad.data for p in model_perb.parameters()]
v = normalization(v)
model_perb.zero_grad()


for lam in lams: 
    model_perb = get_params(model, model_perb, v, lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')
plt.title('Loss landscape perturbed based on gradient direction')

## Example 3: Hessian Trace/Diagonal
We can also use randomized linear algebra to compute Hessian trace or approximate the Hessian diagonal with very little computational overhead. Let's first start with a numpy example, and then we will show results for a NN.

In [ ]:
n = 1000 # the matrix size

# generate the matrix
A = np.random.randn(n, n)
B = A @ A.T

In [ ]:
# Direct get the trace 
trace_B_np = np.matrix.trace(B)
print("The trace of B is: %.4f"%trace_B_np)

We can approximate the above by using Hutchinson Method. It is very similar to power iteration:

$$Tr(B) = \mathbb{E}[v^TBv],$$
$$Diag(B) = \mathbb{E}[v \bigodot Bv].$$

It can be proved that the above expectation converges with smallest variance to the trace if we use Rademacher random numbers (+/-1). In practice you can also use Gaussian random vectors as well.

In [ ]:
# use Hutchinson method to get the trace of B
trace_list = []

for i in range(20):
    v = np.random.randint(2, size=n) 
    v = v.reshape(n, 1) * 2 - 1 # Create Rademacher random numbers
    trace_list.append(v.T @ B @ v)
    trace_B_hutchinson = np.mean(trace_list)
    print("Step %.2d, Current estimated trace: %.1f relative error: %.1e"
          %(i+1, trace_B_hutchinson, (trace_B_hutchinson - trace_B_np) / trace_B_np))

As you can see we can get a very accurate estimate of the trace. Next let's try to approximate the diagonal of B using the matrix-free Hutchinson's method.

In [ ]:
# use Hutchinson method to get the diag of B
diag_est = np.zeros([n, 1])
diag_B_np = np.diag(B)
for i in range(20):
    v = np.random.randint(2, size=n)
    v = v.reshape(n, 1) * 2 - 1
    diag_est += np.multiply(v, (B @ v))
    diag_est_err = np.mean(np.abs(diag_est.reshape(-1) / (i+1) - diag_B_np) / diag_B_np)
    print("Step %.2d, the current average relative error %.1e:"%(i+1,diag_est_err))

Now let's repeate the above for computing the trace and diagonal of Hessian for ResNet20.

In [ ]:

density_eigen, density_weight = hessian_comp.density()

In [ ]:
get_esd_plot(density_eigen, density_weight)

The above ESD plot is very interesting and shows that a lot of the eigenvalues of the Hessian are close to zero. This means that a lot of the directions along the loss landscape is almost flat. We expect this based on the loss landscape that we got above when we used a random direction. Another interesting observation is that there are several large Hessian outliers. The other very interesting finding, is that there are a lot of directions with slight negative curvature. This means that we still have not converged to a perfect local minimum that satisfies first and second order optimality conditions.